In [13]:
import os
import json
from PIL import Image
from tqdm import tqdm

In [14]:
# 폴더를 가져옵니다.
raw_data_folder = r"C:\Users\Smart\Desktop\datas\general_vehicle_full\general_labeling_data"
output_folder = r"C:\Users\Smart\Desktop\code\general_labeling_data"
img_folder = r"C:\Users\Smart\Desktop\datas\general_vehicle_full\general_img"

In [15]:
# 폴더의 파일 목록을 가져옵니다.
files = os.listdir(raw_data_folder)

In [16]:
# 폴더의 파일 목록을 순회합니다.
for file in tqdm(files):
    # 파일 이름을 가져옵니다. 확장자를 제거하여 파일 이름만 남깁니다.
    filename = os.path.splitext(file)[0]

    # 이미지 파일의 경로를 구합니다.
    image_path = os.path.join(img_folder, filename + ".jpg")

    # 이미지 파일의 높이와 너비를 구합니다.
    with Image.open(image_path) as img:
        image_height = img.height
        image_width = img.width

    # 파일을 읽습니다.
    with open(os.path.join(raw_data_folder, file), "r", encoding="UTF-8") as f:
        data = json.load(f)

    # 파일의 키를 이용하여 해당하는 데이터를 가져옵니다.
    file_data = data[list(data.keys())[0]]  # 첫 번째 키값을 가져옵니다.

    labels = []
    shapes = []

    for region_id, region_info in file_data["regions"].items():
        label = region_info["region_attributes"]["class"]
        x1 = region_info["shape_attributes"]["x"]
        y1 = region_info["shape_attributes"]["y"]
        width = region_info["shape_attributes"]["width"]
        height = region_info["shape_attributes"]["height"]
        
        if label == "정상 차량(이륜차 포함)":
            label = "일반차량"
        elif "신호등" in label:
            label = "신호등" 

        x2 = x1 + width
        y2 = y1 + height

        shapes.append({
            "label": label,
            "points": [
                [x1, y1],
                [x2, y2]
            ],
            "group_id": None,
            "description": "",
            "shape_type": "rectangle",
            "flags": {}
        })

    annotation = {
        "version": "5.2.1",
        "flags": {},
        "shapes": shapes,
        "imagePath": filename+".jpg",
        "imageData": None,
        "imageHeight": image_height,
        "imageWidth": image_width
    }

    # 라벨링 데이터를 다른 폴더에 .json형식으로 저장합니다.
    with open(os.path.join(output_folder, filename + ".json"), "w", encoding='UTF-8') as f:
        json.dump(annotation, f, ensure_ascii=False, indent=4)

100%|██████████| 32866/32866 [08:20<00:00, 65.67it/s]
